In [1]:
import os
import sys
import django
import logging
import itertools
import pandas as pd
import numpy as np
import time
from datetime import datetime
from tqdm import tqdm
import django
from django.db.models import Count
from sklearn.metrics import mean_squared_error
sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

from main_site.models import Rating
from title.models import Book
from transaction.models import Master, Detail
import rs_system.recommender as rs
import rs_system.builder.collaborative_filtering_calculator as builder_cf

logger = logging.getLogger('Evaluation')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
transaction_count = Detail.objects.all().count()
test_data_count = round(transaction_count * 10 /100)
print(test_data_count)
book_df = pd.DataFrame(list(Book.objects.all().values()))
# Change transaction_master_db to data frame
transaction_master_df = pd.DataFrame(list(Master.objects.all().values()))
# Change name column
transaction_master_df.rename(columns={'date': 'hire_date'},
                             inplace=True)
# Change transaction_detail_db to data frame
transaction_detail_df = pd.DataFrame(list(Detail.objects.all().values()))
# Join two data frame to one
transaction_df = pd.merge(transaction_master_df, transaction_detail_df, left_on='id', right_on='transaction_id')
transaction_df = pd.merge(transaction_df, book_df[['barcode', 'title_id']], left_on="book_id",
                          right_on="barcode",
                          how="left")
transaction_df = transaction_df.drop(['id_x', 'transaction_id'], axis=1)
transaction_df['return_date'] = transaction_df['return_date'].fillna(datetime.now())
transaction_count_df = transaction_df[['title_id','user_id']].groupby('user_id').agg('count')
transaction_count_df = transaction_count_df.sort_values(by=['title_id'], ascending=False)
transaction_count_df.rename(columns={'title_id': 'count_title_id'},
                                     inplace=True)
transaction_count_df
total_transaction_df = pd.merge(transaction_df, transaction_count_df, left_on="user_id",
                          right_on="user_id")
total_transaction_df = total_transaction_df.sort_values(by=['count_title_id','user_id','hire_date'], ascending=False).reset_index(drop=True)
print("Total Trans",len(total_transaction_df))
total_transaction_df.head(30)

20
Total Trans 198


,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,count_title_id
0,2018-01-29,49,53397-70,2018-07-15,2,93,62620.00,2018-06-12 00:00:00.000000,1,53397-70,55,8
1,2017-10-02,49,58601-80,2018-02-15,2,65,69488.00,2018-01-23 00:00:00.000000,1,58601-80,21,8
2,2017-09-25,49,58604-80,2018-02-15,2,40,100000.00,2018-01-23 00:00:00.000000,1,58604-80,103,8
3,2017-09-25,49,58598-130,2018-02-15,2,41,74538.00,2018-01-23 00:00:00.000000,1,58598-130,56,8
4,2017-09-25,49,58602-80,2018-02-15,2,42,63327.00,2018-01-23 00:00:00.000000,1,58602-80,41,8
5,2017-09-18,49,58604-80,2017-09-25,1,34,25000.00,2017-09-20 00:00:00.000000,1,58604-80,103,8
6,2017-09-18,49,58598-130,2017-09-25,1,35,18634.50,2017-09-20 00:00:00.000000,1,58598-130,56,8
7,2017-09-18,49,58602-80,2017-09-25,1,36,15831.75,2017-09-20 00:00:00.000000,1,58602-80,41,8
8,2018-09-17,40,53394-80,2019-02-15,2,139,100000.00,2019-01-07 00:00:00.000000,1,53394-80,120,8
9,2018-09-15,40,53698-80,2019-02-15,2,138,46864.00,2019-01-14 00:00:00.000000,1,53698-80,12,8


In [4]:
test_user_tile = list()
count = 0
current_row = 0
remove_row = []
while True:
    row = total_transaction_df.iloc[current_row]
    if row.count_title_id == 1:
        break
    if row.count_title_id > 7:
        limit_row = 0
    elif row.count_title_id > 5:
        limit_row = 2
    else:
        limit_row = 1
    for i in range(limit_row):
        remove_row.append(current_row+i)
    title_list = []
    if limit_row != 0:
        test_user_tile.append(dict([('user_id', row.user_id), ('title_id', list(total_transaction_df.iloc[current_row:current_row+limit_row]['title_id']))]))
    current_row = current_row + row.count_title_id
    count += limit_row
    if count == test_data_count:
        break
transaction_df = total_transaction_df.drop(remove_row).reset_index(drop=True)
print("Total tran_df:", len(transaction_df))
print("total_remove",len(remove_row))
transaction_df.head()

Total tran_df: 178
total_remove 20


,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,count_title_id
0,2018-01-29,49,53397-70,2018-07-15,2,93,62620.0,2018-06-12,1,53397-70,55,8
1,2017-10-02,49,58601-80,2018-02-15,2,65,69488.0,2018-01-23,1,58601-80,21,8
2,2017-09-25,49,58604-80,2018-02-15,2,40,100000.0,2018-01-23,1,58604-80,103,8
3,2017-09-25,49,58598-130,2018-02-15,2,41,74538.0,2018-01-23,1,58598-130,56,8
4,2017-09-25,49,58602-80,2018-02-15,2,42,63327.0,2018-01-23,1,58602-80,41,8


In [6]:
def cal_point(sample):
    hire_date_length = abs(sample.return_date - sample.hire_date)
    due_date_length = abs(sample.due_date - sample.hire_date)
    ratio = hire_date_length / due_date_length
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio, 3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1 - ratio) / (ratio - 1))
        sample['point'] = 3 + (7 * ratio)
    sample['point'] = round(sample['point'], 2)
    return sample

In [7]:
transaction_df = transaction_df.groupby('id_y').apply(cal_point).reset_index(drop=True)

# New data frame with user_id, title_id, point
train_data_df = pd.DataFrame()
train_data_df['user_id'] = transaction_df.user_id
train_data_df['title_id'] = transaction_df.title_id
train_data_df['rating'] = transaction_df.point
train_data_df = train_data_df.groupby(['user_id', 'title_id']).mean().round(3).reset_index()
print("Length train:",len(train_data_df))
train_data_df.head()

Length train: 155


,user_id,title_id,rating
0,5,106,8.430
1,6,21,8.440
2,6,50,3.195
3,7,106,8.430
4,8,20,9.450


In [8]:
train_data_df[train_data_df.user_id == 126]

,user_id,title_id,rating
151,126,120,10.0


In [9]:
print(len(test_user_tile))
test_user_tile

19


[{'user_id': 46, 'title_id': [107, 130]},
 {'user_id': 95, 'title_id': [120]},
 {'user_id': 66, 'title_id': [101]},
 {'user_id': 59, 'title_id': [7]},
 {'user_id': 29, 'title_id': [54]},
 {'user_id': 27, 'title_id': [102]},
 {'user_id': 8, 'title_id': [140]},
 {'user_id': 6, 'title_id': [55]},
 {'user_id': 123, 'title_id': [150]},
 {'user_id': 122, 'title_id': [20]},
 {'user_id': 79, 'title_id': [101]},
 {'user_id': 72, 'title_id': [98]},
 {'user_id': 48, 'title_id': [106]},
 {'user_id': 24, 'title_id': [18]},
 {'user_id': 23, 'title_id': [6]},
 {'user_id': 16, 'title_id': [116]},
 {'user_id': 126, 'title_id': [116]},
 {'user_id': 114, 'title_id': [61]},
 {'user_id': 113, 'title_id': [61]}]

In [10]:
rating_cf = builder_cf.CollaborativeFiltering(rating_df=train_data_df)

2019-07-31 15:17:16,988 : INFO : Load all rating data
2019-07-31 15:17:16,995 : INFO : Creating ratings matrix
155it [00:00, 327845.24it/s]


In [11]:
rating_predict_df = rating_cf.predict_all_item(save = False)

100%|██████████| 130/130 [02:37<00:00,  1.07s/it]


In [12]:
rating_predict_df.head(5)

,user_id,title_id,rating,type
0,5,12,6.732778,ib_predicted
1,5,54,5.318889,ib_predicted
2,5,63,8.606111,ib_predicted
3,5,88,7.598611,ib_predicted
4,5,97,6.000397,ib_predicted


In [18]:
count = 0
count_same_in_hybrid = 0
count_same_in_cf = 0
precision = 0
for item in tqdm(test_user_tile):
    book_id_list = []
    rec_list_cf = rs.get_top_recs_using_collaborative_filtering(user_index=item['user_id'],rating_df=rating_predict_df)
    rec_list_hybird = rs.get_top_recs_using_hybrid(user_index=item['user_id'],rating_df=rating_predict_df)
    rec_list_hybird = np.array(rec_list_hybird.title_id,dtype=np.int)
    print('User_ID: ',item['user_id'])
    print(item['title_id'])
    print('Suggestion List Hybird:',rec_list_hybird)
    same_in_hybird = list(set(item['title_id']) & set(rec_list_hybird))
    print('Same in Hybird:',same_in_hybird)
    if len(same_in_hybird) > 0:
        count_same_in_hybrid +=1
        if same_in_hybird[0] == rec_list_hybird[0]:
            precision += 1
#     if rec_list_cf is not None:
#         count+=1
#         rec_list_cf = np.array(rec_list_cf.title_id,dtype=np.int)
#         same_in_cf = list(set(item['title_id']) & set(rec_list_cf))
#         if len(same_in_cf) > 0:
#             count_same_in_cf +=1

#         print('Suggestion List Collaborative Filterting:',rec_list_cf)
#         print('Same in CF:',same_in_cf)
    print("=============")
# print("Can predict:",count)
print("Same in hybird:",count_same_in_hybrid)
print("precision:",precision)
# print("Same in cf:",count_same_in_cf)

 26%|██▋       | 5/19 [00:00<00:00, 18.59it/s]

User_ID:  46
[107, 130]
Suggestion List Hybird: [ 13  14  41  54  82  88  96 100 102 107 108 136]
Same in Hybird: [107]
User_ID:  95
[120]
Suggestion List Hybird: [ 32  61  62  64  65  67 105 120 122 124 131 150]
Same in Hybird: [120]
User_ID:  66
[101]
Suggestion List Hybird: [15 16 39 45 60 77 79 96 97 99]
Same in Hybird: []
User_ID:  59
[7]
Suggestion List Hybird: [ 37  41  73  96  97 106 108 138 140]
Same in Hybird: []
User_ID:  29
[54]
Suggestion List Hybird: [  1  10  14  41  82  88 107 108 130 137]
Same in Hybird: []


 47%|████▋     | 9/19 [00:00<00:00, 18.15it/s]

User_ID:  27
[102]
Suggestion List Hybird: [  4  11  17  21  22  40  41  55  56  88 102 152]
Same in Hybird: [102]
User_ID:  8
[140]
Suggestion List Hybird: [  7  41  42  73  74  84  88  96 100 102 106 143]
Same in Hybird: []
User_ID:  6
[55]
Suggestion List Hybird: [ 21  22  41  43  49  50  55  56  96 136 139 148]
Same in Hybird: [55]
User_ID:  123
[150]
Suggestion List Hybird: [ 14  32  41 106 107 108 112 120 121 122 124 154]
Same in Hybird: []


 68%|██████▊   | 13/19 [00:00<00:00, 17.19it/s]

User_ID:  122
[20]
Suggestion List Hybird: [ 12  42  57  60  66  88  96  97 100 106 120 138]
Same in Hybird: []
User_ID:  79
[101]
Suggestion List Hybird: [39 77 96 97 99]
Same in Hybird: []
User_ID:  72
[98]
Suggestion List Hybird: [ 11  34  35  41  42  43  73  96  97  98  99 117]
Same in Hybird: [98]
User_ID:  48
[106]
Suggestion List Hybird: [  1  10  13  14  23  41  54  82  88  97 107 108]
Same in Hybird: []


 79%|███████▉  | 15/19 [00:00<00:00, 17.01it/s]

User_ID:  24
[18]
Suggestion List Hybird: [ 19  24  37  41 106 107 108 112 129 147 149 164]
Same in Hybird: []
User_ID:  23
[6]
Suggestion List Hybird: [ 23  24  31  35  53  70  71  80 100 146 147 155]
Same in Hybird: []
User_ID:  16
[116]
Suggestion List Hybird: [ 12  41  57  60  66  84  97 100 106 110 116 119]
Same in Hybird: [116]


100%|██████████| 19/19 [00:01<00:00, 16.11it/s]

User_ID:  126
[116]
Suggestion List Hybird: [  1  12  32  41  88  97 100 106 107 108 116 121]
Same in Hybird: [116]
User_ID:  114
[61]
Suggestion List Hybird: [ 23  60  61  62  63  76  91 120 154]
Same in Hybird: [61]
User_ID:  113
[61]
Suggestion List Hybird: [ 23  60  61  62  63  76  91 120 154]
Same in Hybird: [61]
Same in hybird: 9
precision: 0
